# Testing Model1

Testing recommender systems is less intuitive than testing other predictive models. There are many common metrics to use, which can be found [here in this useful Medium article](https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832). The idea for all of these methods is similar: generate a list of $n$ top ranked items to show the user, and your score is based on how many of the items you recommend are relevant to the user. The word "relevant" is not well-defined, but normally means that the user has interacted with this item in the past. 

In our case, our items are the restaurants and the interactions are orders. One issue with our data is that between 30-40% of our users have only ordered from a single restaurant, which makes the above methods tricky/impossible to apply.

Recall that our model takes in a user's past five (5) orders as inputs together with one restaurant R and tries to predict how likely the user will order from R next. We will evaluate our model as follows. Given a user's sequence of five restaurants, and given the target restaurant R which is next in the sequence, we use the model to generate a ranked list of $k=5$ more restaurants that it thinks is the most likely to be ordered from next. If R is among the $k$ generated restaurants, we add $+1$ to the score, otherwise $+0$. 

Since there are only $100$ restaurants to recommend from, it should be very feasible to simply iterate through this list to score each one, sort the rankings, and slice the top 5 as recommendations. This is a very privillaged position we are in, since many recommender systems are deployed in a context where there are millions of items to recommend from. In that case, one could perform clustering based on the customer and vendor embeddings, and then perform ranking within the clusters.  

Each user in the test set may possibly geenerate many length $5+1$ sequences, and we shall use all of these during the evaluation process. In the deployed version of the model, we could simply give recommendations based on the last $5$ orders made by the user.

Baselines:
1) Recommend the $5$ most popular restaurants.
2) Given a sequence of $5$ orders, $m$ of which are unique and not null, recommend those restaurants together with the $5-m$ most popular restaurants. 

We will define 'popularity' of a vendor to be equal to its number of orders in the training data. Note that the unordered set of top five most popular vendors would be the same if we changed the definition of popularity of a vendor to be equal to its number of unique customers (although their rankings are slightly different, see the last cell of "Munging.ipynb"). 

In [3]:
import pandas as pd
import pickle
import torch
from PreprocessingHelpers import CustomDataset
from torch.utils.data import Dataset, DataLoader
from Models.Models import Model1


## Load Test Data

In [2]:
with open("ProcessedData/test_sequences_padded_dataset.pkl", "rb") as file:
    test_sequences_padded_dataset = pickle.load(file)

AttributeError: Can't get attribute 'CustomDataset' on <module '__main__'>

## Define Baselines

In [6]:
with open("ProcessedData/popular_vendors.pkl", "rb") as file:
    popular_vendors = pickle.load(file)

popular_vendors.head(10)

id
28    3237
25    2717
14    2681
19    2453
18    2184
15    2159
75    1377
21    1274
36    1145
6     1078
Name: num_orders, dtype: int64

In [8]:
most_popular = popular_vendors[:5].index.tolist()
most_popular

[28, 25, 14, 19, 18]